Todo:

- Zpozdeni!
- udělat to jako skript, co bere vše neoscrapované
- bez Kč a km (rozdělit)

Sólo sešit:
- mazat oscrapované soubory

In [185]:
import os
import re
from datetime import datetime
import json

In [275]:
with open(os.path.join("downloads/2024-11-01","cd_Brno_Hamburg_D07_2024-11-01_08-57-29.txt"), "r", encoding="utf-8") as file:
    raw_html = file.read()

In [277]:
spojeni = raw_html.split("""<article""")
spojeni = spojeni[1:]

In [59]:
len(spojeni)

35

In [279]:
pokus = spojeni[10].splitlines()

In [281]:
pokus

[' class="overview-connection" data-bind="css: {\'overview-connection--beige\': hasHistoryTrain() || (connList.isBack() &amp;&amp; connThere != null &amp;&amp; connThere.hasHistoryTrain()), \'noprint\': $parent.isAnyConnectionSelected() &amp;&amp; !connectionSelected()}">',
 '                <h2 class="vh">Spojení</h2>',
 '                ',
 '',
 '',
 '<div class="overview-schedule--simple" data-bind="visible: !detailVisible(), attr: {id: \'os-simple\' + id}" id="os-simple57586874">',
 '    <div class="overview-schedule--simple__content">',
 '        <ul class="schedule__labels unlist">',
 '            <!-- ko foreach: trains -->',
 '            <li class="schedule__label">',
 '                <div data-bind="css: delayCss()" class="detail-link label label--mobile-small label--with-icon label--with-tag overview-label label--gray">',
 '                    <p class="label__text">',
 '                        <span class="icon icon-train" data-bind="css: vehicleIcon"></span>',
 '         

In [283]:
with open("pokus.txt", "w+", encoding="utf-8") as pokus_file:
    pokus_file.write("\n".join(pokus))

In [269]:
def oscrapuj_cd(slozka, soubor):
    
    with open(os.path.join(slozka, soubor), "r", encoding="utf-8") as file:
        raw_html = file.read()
    spojeni = raw_html.split("""<article""")
    spojeni = spojeni[1:]
    slovniky = []
    for s in spojeni:
        s = s.splitlines()
        slovnik = {}
        slovnik['prodejce'] = "ČD"
        slovnik['vlaky'], slovnik['obsazenost'] = [], []
        oscrapovano = re.search(r'\d{4}-\d{2}-\d{2}_\d{2}-\d{2}-\d{2}', soubor).group(0)
        slovnik['oscrapovano'] = datetime.strptime(oscrapovano, "%Y-%m-%d_%H-%M-%S")
        den = False
        for radek in s:
            if "text: to(lastTrain($index()))" in radek:
                slovnik['kam'] = radek.split('>')[-2].split('<')[0]
            if "text: from(firstTrain($index()))" in radek:
                slovnik['odkud'] = radek.split('>')[-2].split('<')[0]
            if ("text: journeyDateTextFrom()" in radek) and (den == False):
                den = radek.split('>')[-2].split('<')[0]
            if "text: depTime(firstTrain($index()))" in radek:
                cas_odjezdu = re.search(r"\d{1,2}:\d\d", radek).group(0)
            # if "text: arrTime(lastTrain($index()))" in radek:
            #    slovnik['cas_prijezdu'] = re.search(r"\d{1,2}:\d\d", radek).group(0)
            if "text: buyButtonText()" in radek:
                slovnik['cena'] = radek.split('-->')[1].split('<!--')[0]
            if "text: model.trainTypeAndNum" in radek:
                if radek.split('-->')[1].split('<!--')[0] not in slovnik['vlaky']:
                    slovnik['vlaky'].append(radek.split('-->')[1].split('<!--')[0])
            if """<span aria-hidden="true" data-bind="visible: occupancyLevelFull(), text: occupancyLevelText()" style="display: none;">""" in radek:
                slovnik['obsazenost'].append(radek.split('>')[-2].split('<')[0])
            if """<span data-bind="text: timeLength, visible: timeLength != null &amp;&amp; timeLength != ''">""" in radek:
                slovnik['jizdni_doba'] = radek.split('>')[-2].split('<')[0]
            if """<span data-bind="text: distance, visible: distance != null &amp;&amp; distance != ''" class="mobile-hidden">""" in radek:
                slovnik['vzdalenost'] = radek.split('>')[-2].split('<')[0]
            if """<span class="icon icon-bus" data-bind="ifnot: isLegend, visible: !isLegend &amp;&amp; icoSrc, css: icoSrc, attr: {title: desc, 'aria-label': desc}" title="Náhradní autobusová doprava" aria-label="Náhradní autobusová doprava"></span>""" in radek:
                slovnik['autobus'] = True
            if """OMEZENÍ NA TRASE""" in radek:
                slovnik['omezeni'] = True
        slovnik['den'] = den.split(' ')[0].lower().strip()
        slovnik['odjezd'] = datetime.strptime('-'.join(den.split(' ')[1].split('.')[::-1]) + " " + cas_odjezdu, "%Y-%m-%d %H:%M")
        slovnik['predstih'] = slovnik['odjezd'] - slovnik['oscrapovano']
        if len(slovnik['vlaky']) == 1:
            slovnik['vlaky'] = slovnik['vlaky'][0] 
            try:
                slovnik['obsazenost'] = slovnik['obsazenost'][0] ## todo: obsazenost se přidává n+1×, toto je laciný workaround pro většinu případů 
            except:
                pass
        slovniky.append(slovnik)
    return slovniky

In [261]:
for x in os.listdir("downloads"):
    kam = f"data_raw/{x}" 
    os.makedirs(kam, exist_ok=True)
    for y in os.listdir(f"downloads/{x}"):
        konverze = oscrapuj_cd(f"downloads/{x}",y)
        nazev_souboru = y.replace('txt','json')
        print(nazev_souboru)
        with open(os.path.join(kam, nazev_souboru), 'w+', encoding='utf-8') as export:
            export.write(json.dumps(konverze, sort_keys=True, default=str))

cd_Brno_Hamburg_D00_2024-10-31_23-32-19.json
cd_Brno_Hamburg_D01_2024-10-31_23-35-33.json
cd_Brno_Hamburg_D02_2024-10-31_23-39-05.json
cd_Brno_Hamburg_D03_2024-10-31_23-42-37.json
cd_Brno_Hamburg_D04_2024-10-31_23-46-17.json
cd_Brno_Hamburg_D05_2024-10-31_23-50-13.json
cd_Brno_Hamburg_D06_2024-10-31_23-54-11.json
cd_Brno_Hamburg_D07_2024-10-31_23-58-19.json
cd_Brno_Praha_D00_2024-10-31_23-30-46.json
cd_Brno_Praha_D01_2024-10-31_23-33-55.json
cd_Brno_Praha_D02_2024-10-31_23-37-22.json
cd_Brno_Praha_D03_2024-10-31_23-40-51.json
cd_Brno_Praha_D04_2024-10-31_23-44-26.json
cd_Brno_Praha_D05_2024-10-31_23-48-16.json
cd_Brno_Praha_D06_2024-10-31_23-52-12.json
cd_Brno_Praha_D07_2024-10-31_23-56-13.json
cd_Liberec_Salzburg_D00_2024-10-31_23-32-44.json
cd_Liberec_Salzburg_D01_2024-10-31_23-36-00.json
cd_Liberec_Salzburg_D02_2024-10-31_23-39-31.json
cd_Liberec_Salzburg_D03_2024-10-31_23-43-06.json
cd_Liberec_Salzburg_D04_2024-10-31_23-46-47.json
cd_Liberec_Salzburg_D05_2024-10-31_23-50-44.json
cd

In [273]:
oscrapuj_cd("downloads/2024-11-01","cd_Brno_Hamburg_D07_2024-11-01_08-57-29.txt")

[{'prodejce': 'ČD',
  'vlaky': ['EC 280 Metropolitan', 'EC 378 Berliner', 'ICE 506'],
  'obsazenost': ['Předpokládané vysoké obsazení - doporučujeme včasné pořízení místenky',
   'Předpokládané vysoké obsazení - doporučujeme včasné pořízení místenky',
   'Předpokládané střední obsazení - doporučujeme včasné pořízení místenky',
   'Téměr vyprodaná místa k sezení'],
  'oscrapovano': datetime.datetime(2024, 11, 1, 8, 57, 29),
  'odkud': 'Brno hl.n.',
  'kam': 'Hamburg Hbf',
  'jizdni_doba': '10:32 hod.',
  'omezeni': True,
  'cena': '2425 Kč',
  'den': 'pátek',
  'odjezd': datetime.datetime(2024, 11, 8, 9, 39),
  'predstih': datetime.timedelta(days=7, seconds=2491)},
 {'prodejce': 'ČD',
  'vlaky': ['EC 172 Hungaria', 'ICE 504'],
  'obsazenost': ['Předpokládané vysoké obsazení - doporučujeme včasné pořízení místenky',
   'Předpokládané vysoké obsazení - doporučujeme včasné pořízení místenky',
   'Téměr vyprodaná místa k sezení'],
  'oscrapovano': datetime.datetime(2024, 11, 1, 8, 57, 29),
